# Inicializar

In [1]:
import gurobipy as gp
from gurobipy import GRB
import os
import re
import pandas as pd

In [2]:
# Mostrar el modelo
def displayModel(model,name):
  try:
      os.remove(name)
  except OSError:
      pass
  model.write(name) # Escribir el modelo a un archivo .lp
  model.display()

# Modelo base

In [3]:
def base_model(N):
    if N < 4:
        raise ValueError("El modelo híbrido requiere al menos 4 nodos (mínimo 3 paralelo, 1 serie).")

    model = gp.Model(f"General_Model_{N}_Nodos")

    # Parámetros
    I = range(3) # 0: Low, 1: Medium, 2: High
    U = range(N) # Nodos a desplegar
    W = range(2) # 0: Serie, 1: Paralelo

    L = 10
    c_values = {0: 5, 1: 12, 2: 20}

    # Variables
    x = model.addVars(U, I, W, vtype=GRB.BINARY, name="x") # Variable de decisión

    # Variables auxiliares
    N_s = model.addVar(vtype=GRB.INTEGER, name="N_s") # Número de nodos en serie
    N_p = model.addVar(vtype=GRB.INTEGER, name="N_p") # Número de nodos en paralelo
    z = model.addVar(vtype=GRB.INTEGER, name="z") # Variable auxiliar
    delta_Np = model.addVar(vtype=GRB.BINARY, name="delta_Np") # Variable binaria delta de kronecker

    """Restricciones comunes"""

    # Restricciones de unicidad para i y w
    model.addConstrs(
        (gp.quicksum(x[u, i, w] for i in I for w in W) == 1 for u in U), name="Unicidad_iw"
    )

    model.addConstr(N_s == gp.quicksum(x[u, i, 0] for i in I for u in U), name="Ns_def") # Número de nodos en serie
    model.addConstr(N_p == gp.quicksum(x[u, i, 1] for i in I for u in U), name="Np_def") # Número de nodos en paralelo
    model.addConstr(z * 2 == N_p * (N_p - 1), name="Quadratic_Exact") # Restricción cuadrática exacta

    # Restricción del delta de Kronecker: Si Np == 0 → delta_Np = 1, sino delta_Np = 0
    model.addGenConstrIndicator(delta_Np, True, N_p == 0, name="Kronecker_1")
    model.addGenConstrIndicator(delta_Np, False, N_p >= 1, name="Kronecker_0")

    # 🔴 Esta restricción solo aplica en el modelo híbrido
    model.addConstr(N_p >= 3, name="Np_Condition") # Número de nodos en paralelo mínimo

    NodesCost = gp.quicksum(c_values[i] * x[u, i, w] for i in I for u in U for w in W) # Costo de los nodos desplegados
    LinksCost = L * (N_s + z - delta_Np) # Costo de los enlaces desplegados
    # LinksCost = 0
    model.setObjective(NodesCost + LinksCost, GRB.MINIMIZE)

    #####

    # displayModel(model, "General_Model.lp") # Mostrar el modelo
    model.setParam('OutputFlag', 0)  # No mostrar salida por defecto de Gurobi

    model.optimize()

    if model.status == GRB.OPTIMAL:
        variablesDecision = dict()
        for var in model.getVars():
            variablesDecision[var.varName] = var.x
        return model.objVal, variablesDecision, model
    else:
        raise Exception("No se encontró una solución óptima.")

# Modelos heredados

In [4]:
def serie_model(model_base, N):
    if N < 2:
        raise ValueError("El modelo solo serie requiere al menos 2 nodos.")

    model_base.update()
    model = model_base.copy()

    model.remove(model.getConstrByName("Np_Condition"))  # 🚨 Remover restricción híbrida

    N_p = model.getVarByName("N_p")  # Obtener variable N_p del modelo base
    if N_p is not None:
        model.addConstr(N_p == 0, name="Solo_Serie")
    else:
        print("La variable N_p no se encontró en el modelo copiado.")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        variablesDecision = dict()
        for var in model.getVars():
            variablesDecision[var.varName] = var.x
        return model.objVal, variablesDecision
    else:
        raise Exception("No se encontró una solución óptima.")

In [5]:
def parallel_model(model_base, N):
    if N < 3:
        raise ValueError("El modelo solo paralelo requiere al menos 3 nodos.")

    model_base.update()
    model = model_base.copy()

    N_s = model.getVarByName("N_s")  # Obtener variable N_s del modelo base
    if N_s is not None:
        model.addConstr(N_s == 0, name="Solo_Paralelo")
    else:
        print("La variable N_s no se encontró en el modelo copiado.")

    model.optimize()

    if model.status == GRB.OPTIMAL:
        variablesDecision = dict()
        for var in model.getVars():
            variablesDecision[var.varName] = var.x
        return model.objVal, variablesDecision
    else:
        raise Exception("No se encontró una solución óptima.")

# Pruebas

In [6]:
try:
    costoHibrido, variablesDecisionHibrido, modeloHibrido = base_model(4)
    print("Modelo híbrido")
    print(f"Objetivo: {costoHibrido}")
    print("Variables de decisión:")
    for varName, value in variablesDecisionHibrido.items():
        print(f"{varName}: {value}")
    costoSerie, variablesDecisionSerie = serie_model(modeloHibrido, 4)
    print("\nModelo solo serie")
    print(f"Objetivo: {costoSerie}")
    print("Variables de decisión:")
    for varName, value in variablesDecisionSerie.items():
        print(f"{varName}: {value}")
    costoParalelo, variablesDecisionParalelo = parallel_model(modeloHibrido, 4)
    print("\nModelo solo paralelo")
    print(f"Objetivo: {costoParalelo}")
    print("Variables de decisión:")
    for varName, value in variablesDecisionParalelo.items():
        print(f"{varName}: {value}")
except Exception as e:
    print(f"Error: {e}")

Restricted license - for non-production use only - expires 2026-11-23
Modelo híbrido
Objetivo: 60.0
Variables de decisión:
x[0,0,0]: 1.0
x[0,0,1]: 0.0
x[0,1,0]: 0.0
x[0,1,1]: 0.0
x[0,2,0]: 0.0
x[0,2,1]: 0.0
x[1,0,0]: 0.0
x[1,0,1]: 1.0
x[1,1,0]: 0.0
x[1,1,1]: 0.0
x[1,2,0]: 0.0
x[1,2,1]: 0.0
x[2,0,0]: 0.0
x[2,0,1]: 1.0
x[2,1,0]: 0.0
x[2,1,1]: 0.0
x[2,2,0]: 0.0
x[2,2,1]: 0.0
x[3,0,0]: 0.0
x[3,0,1]: 1.0
x[3,1,0]: 0.0
x[3,1,1]: 0.0
x[3,2,0]: 0.0
x[3,2,1]: 0.0
N_s: 1.0
N_p: 3.0
z: 3.0
delta_Np: 0.0

Modelo solo serie
Objetivo: 50.0
Variables de decisión:
x[0,0,0]: 1.0
x[0,0,1]: 0.0
x[0,1,0]: 0.0
x[0,1,1]: 0.0
x[0,2,0]: 0.0
x[0,2,1]: 0.0
x[1,0,0]: 1.0
x[1,0,1]: 0.0
x[1,1,0]: 0.0
x[1,1,1]: 0.0
x[1,2,0]: 0.0
x[1,2,1]: 0.0
x[2,0,0]: 1.0
x[2,0,1]: 0.0
x[2,1,0]: 0.0
x[2,1,1]: 0.0
x[2,2,0]: 0.0
x[2,2,1]: 0.0
x[3,0,0]: 1.0
x[3,0,1]: 0.0
x[3,1,0]: 0.0
x[3,1,1]: 0.0
x[3,2,0]: 0.0
x[3,2,1]: 0.0
N_s: 4.0
N_p: 0.0
z: 0.0
delta_Np: 1.0

Modelo solo paralelo
Objetivo: 80.0
Variables de decisión:
x[0,0,0]: 